## Problem Statement

The dataset is similar to MNIST, but includes images of certain clothing and accessory. The objective is to add some noise to the images and then, use an Autoencoder to desnoise images.

### Import libraries

In [ ]:
import tensorflow as tensorflow
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, Activation,BatchNormalization
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import pandas as pd
import numpy as np
from tensorflow.keras.layers import Dense, Dropout, Input,LeakyReLU,InputLayer,Conv2DTranspose,MaxPool2D,UpSampling2D, Reshape
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.datasets import fashion_mnist
from tqdm import tqdm
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K

## Analyse and explore the data

In [ ]:
(x_train,y_train),(x_test,y_test)=fashion_mnist.load_data()

### Check the shape of the dataset

In [ ]:
print("shape of training data",x_train.shape)

In [ ]:
print("shape of testing data",x_test.shape)

### Perform Null values testing

In [ ]:
if np.isnan(x_train).any():
  print("Null values are present in train dataset")
else:
  print("Null values are not present in train dataset")

In [ ]:
if np.isnan(x_test).any():
  print("Null values are present in test dataset")
else:
  print("Null values are not present in test dataset")

### Visualize the images present in the dataset

In [ ]:
def display(array1):
    """ Displays any five random images from the given dataset"""

    n = 5

    indices = np.random.randint(len(array1), size=n)
    images1 = array1[indices, :]

    plt.figure(figsize=(10, 4))
    for i,image1 in enumerate(images1):
    
      ax = plt.subplot(1,n,i+1)
      plt.imshow(image1.reshape(28, 28))
      plt.gray()
      ax.get_xaxis().set_visible(False)
      ax.get_yaxis().set_visible(False)
    plt.show()

In [ ]:
display(x_train)

### Find out the unique count of each category

In [ ]:
unique, counts = np.unique(y_train, return_counts=True)
dict(zip(unique, counts))

## Data Preprocessing

### Normalize all the value between 0 and 1

In [ ]:
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1).astype('float32')
x_train = x_train/255
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1).astype('float32')
x_test = x_test/255

### Adding noise to train and test images

In [ ]:
noise_factor = 0.5
train_noisy= x_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape)
train_noisy=np.clip(train_noisy, 0.0, 1.0)

##test images
test_noisy= x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape)
test_noisy=np.clip(test_noisy, 0.0, 1.0)

### Visualizing the noisy data

In [ ]:
display(train_noisy)

## Building an Autoencoder

In [ ]:
input_layer=Input(shape=(28,28,1))
encoder_layer1=Conv2D(16,(3,3),padding='same',activation='relu')(input_layer)
max_pool1=MaxPooling2D((2, 2))(encoder_layer1)
encoder_layer2=Conv2D(32,(3,3),padding='same',activation='relu')(max_pool1)
maxpool_layer2=MaxPooling2D((2, 2))(encoder_layer2)
encoder_layer3=Conv2D(64,(3,3),padding='same')(maxpool_layer2)


## Decoder

de_layer1=Conv2DTranspose(64,(3,3),padding='same',activation='relu')(encoder_layer3)
upsample_layer1=UpSampling2D((2,2))(de_layer1)
de_layer2=Conv2DTranspose(32,(3,3),padding='same',activation='relu')(upsample_layer1)
upsample_layer2=UpSampling2D((2,2))(de_layer2)
de_layer3=Conv2DTranspose(16,(3,3),padding='same',activation='relu')(upsample_layer2)

de_layer4=Conv2D(1,(3,3),activation='sigmoid',padding='same')(de_layer3)

autoencoder = Model(input_layer, de_layer4)
autoencoder.compile(optimizer="adam", loss="binary_crossentropy",metrics=['accuracy'])
autoencoder.summary()

In [ ]:
history=autoencoder.fit(train_noisy,x_train,epochs=10,batch_size=32,shuffle=True,validation_data=(test_noisy, x_test))

## Plot the training history

In [ ]:
from matplotlib import pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
from matplotlib import pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

## Evaluate the model performance

In [ ]:
score=autoencoder.evaluate(test_noisy,x_test)

In [ ]:
print('Test loss:', score[0]) 
print('Test accuracy:', score[1])

## Visualizedthe Denoised ImagesUsing Matplotlib

In [ ]:
pred=autoencoder.predict(test_noisy)

In [ ]:
display(pred)

## Summary

* Download the dataset form keras. It consists images of t-shirts,pullovers, bags,trousers,dresses, ankleboots etc upto 10 classes.
* Visualize the images in the dataset.
* Then add noise to the images which are used for training the autoencoder. Salt and pepper noise to the images.
* Then build the autoencoder model. Encoder is used to compress the images and decoder is used extract the images from the compressed images.
* Now the noisy images generated are fed to the model. The encoder compresses these images.
* Where as the decoder tries to predict the original images without noise. The noisy images are given as samples and the original images are given as labels.
* This is how the model gets trained and when we give any noisy image it produces the denoised images.